In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [5]:
import os
from ultralytics import YOLO
import glob
import cv2
import math
import torch
import shutil
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


<h1>Fine Tuning YOLOv11 on Raw Local Data</h1>

In [13]:
# Define paths
source_images_dir = "/kaggle/input/satellite-imagery-lahore-dha-phase-6/Dataset/train/images"
source_labels_dir = "/kaggle/input/satellite-imagery-lahore-dha-phase-6/Dataset/train/labels"
yolo_dataset_dir = "/kaggle/working/local_raw_yolo"

# Create directories
os.makedirs(f"{yolo_dataset_dir}/images/train", exist_ok=True)
os.makedirs(f"{yolo_dataset_dir}/labels/train", exist_ok=True)

# Get and sort image files
image_files = sorted([f for f in os.listdir(source_images_dir) if f.endswith('.png')])
label_files = [f.replace('.png', '_label.txt') for f in image_files]

# Verify label files exist
missing_labels = [lbl for lbl in label_files if not os.path.exists(os.path.join(source_labels_dir, lbl))]
if missing_labels:
    print(f"⚠️ Warning: {len(missing_labels)} label files missing!")
    print("First 5 missing:", missing_labels[:5])

# Process and rename files sequentially
for idx, (img_file, lbl_file) in enumerate(zip(image_files, label_files), start=1):
    # New standardized names
    new_img_name = f"image_{idx}.bmp"
    new_lbl_name = f"image_{idx}.txt"
    
    # Convert and save image as BMP
    img_path = os.path.join(source_images_dir, img_file)
    new_img_path = os.path.join(yolo_dataset_dir, "images/train", new_img_name)
    
    with Image.open(img_path) as img:
        img.save(new_img_path, format='BMP')
    
    # Copy and rename label file
    lbl_path = os.path.join(source_labels_dir, lbl_file)
    new_lbl_path = os.path.join(yolo_dataset_dir, "labels/train", new_lbl_name)
    
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, new_lbl_path)
    else:
        print(f"⚠️ Missing label: {lbl_file} for image {img_file}")

# Create YAML config
data_yaml = f"""
path: {yolo_dataset_dir}
train: images/train
val: images/train  # using same set for validation to satisfy YOLO
nc: 1
names: ['solar_panel']
"""

with open(os.path.join(yolo_dataset_dir, "data.yaml"), "w") as f:
    f.write(data_yaml.strip())

print("\n✅ Dataset preparation complete!")
print(f"Processed {len(image_files)} images")
print(f"Standardized naming to image_X.bmp/image_X.txt format")
print(f"Output directory: {yolo_dataset_dir}")


✅ Dataset preparation complete!
Processed 300 images
Standardized naming to image_X.bmp/image_X.txt format
Output directory: /kaggle/working/local_raw_yolo


In [14]:
if torch.cuda.is_available():
    print(f"CUDA is available. Total devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

CUDA is available. Total devices: 2
Device 0: Tesla T4
Device 1: Tesla T4


In [15]:
model = YOLO("yolo11l.pt")
model.train(data = "/kaggle/working/local_raw_yolo/data.yaml", epochs = 20, imgsz = 640, device = [0, 1], val = False)

Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/local_raw_yolo/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning /kaggle/working/local_raw_yolo/labels/train... 300 images, 93 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 2063.27it/s]

train: New cache created: /kaggle/working/local_raw_yolo/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 1351.9±1164.0 MB/s, size: 23828.2 KB)


val: Scanning /kaggle/working/local_raw_yolo/labels/train.cache... 300 images, 93 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


                   all        300        839      0.595      0.563      0.582      0.266


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 18.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f511eac0190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

<h1>Fine Tuning YOLOv11 pre-trained on High Resolution Data (Zenodo)</h1>

In [17]:
model = YOLO("/kaggle/input/yolov11ft_zenodo/pytorch/default/1/YOLOv11_Zenodo_best.pt") #Using pre-trained weights
model.train(data = "/kaggle/working/local_raw_yolo/data.yaml", epochs = 20, imgsz = 640, device = [0, 1], val = False) #same hyperparameters

Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/local_raw_yolo/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/yolov11ft_zenodo/pytorch/default/1/YOLOv11_Zenodo_best.pt, momentum=0.937, mosaic=1.0, multi_scale=False,

train: Scanning /kaggle/working/local_raw_yolo/labels/train.cache... 300 images, 93 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1347.7±319.5 MB/s, size: 23828.2 KB)


val: Scanning /kaggle/working/local_raw_yolo/labels/train.cache... 300 images, 93 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


                   all        300        839      0.645      0.601      0.665      0.346


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 18.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f511266ee90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

<h1>Running Inferences on YOLOv11 Fine-Tuned on Local Data Only</h1>

In [19]:
test_source_dir = "/kaggle/input/satellite-imagery-lahore-dha-phase-6/Dataset/test/images"
working_test_dir = "/kaggle/working/test_samples"
os.makedirs(working_test_dir, exist_ok=True)

print("🔄 Converting test images to BMP...")
test_images = [f for f in os.listdir(test_source_dir) if f.endswith('.png')]

for img_file in test_images:
    src_path = os.path.join(test_source_dir, img_file)
    dest_path = os.path.join(working_test_dir, os.path.splitext(img_file)[0] + '.bmp')
    
    with Image.open(src_path) as img:
        img.save(dest_path, format='BMP')

print(f"✅ Converted {len(test_images)} test images to BMP format")

# 3. Load trained model
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")  # Or your custom model path

# 4. Run inference on all test images
print("\n🔍 Running detection on test images...")
results = model.predict(
    source=working_test_dir,
    conf=0.25,  # Confidence threshold
    iou=0.45,   # NMS IoU threshold
    imgsz=640,   # Inference size
    save=True,   # Save results
    save_txt=True,  # Save results as .txt
    save_conf=True,  # Save confidence scores
    project="/kaggle/working/runs/detect",  # Save location
    name="predict",  # Subfolder name
    exist_ok=True  # Overwrite existing files
)

print("\n🎯 Detection complete! Results saved to:")
print("/kaggle/working/runs/detect/predict")

🔄 Converting test images to BMP...
✅ Converted 30 test images to BMP format

🔍 Running detection on test images...

image 1/30 /kaggle/working/test_samples/exported_grid_test_1.bmp: 640x640 1 solar_panel, 47.3ms
image 2/30 /kaggle/working/test_samples/exported_grid_test_10.bmp: 640x640 2 solar_panels, 47.3ms
image 3/30 /kaggle/working/test_samples/exported_grid_test_11.bmp: 640x640 3 solar_panels, 46.2ms
image 4/30 /kaggle/working/test_samples/exported_grid_test_12.bmp: 640x640 2 solar_panels, 31.4ms
image 5/30 /kaggle/working/test_samples/exported_grid_test_13.bmp: 640x640 3 solar_panels, 31.4ms
image 6/30 /kaggle/working/test_samples/exported_grid_test_14.bmp: 640x640 1 solar_panel, 31.1ms
image 7/30 /kaggle/working/test_samples/exported_grid_test_15.bmp: 640x640 (no detections), 31.2ms
image 8/30 /kaggle/working/test_samples/exported_grid_test_16.bmp: 640x640 4 solar_panels, 26.7ms
image 9/30 /kaggle/working/test_samples/exported_grid_test_17.bmp: 640x640 6 solar_panels, 25.7ms
imag

In [20]:
test_source_dir = "/kaggle/input/satellite-imagery-lahore-dha-phase-6/Dataset/test/images"
working_test_dir = "/kaggle/working/test_samples"
os.makedirs(working_test_dir, exist_ok=True)

print("🔄 Converting test images to BMP...")
test_images = [f for f in os.listdir(test_source_dir) if f.endswith('.png')]

for img_file in test_images:
    src_path = os.path.join(test_source_dir, img_file)
    dest_path = os.path.join(working_test_dir, os.path.splitext(img_file)[0] + '.bmp')
    
    with Image.open(src_path) as img:
        img.save(dest_path, format='BMP')

print(f"✅ Converted {len(test_images)} test images to BMP format")

# 3. Load trained model
model = YOLO("/kaggle/working/runs/detect/train3/weights/best.pt")  # Or your custom model path

# 4. Run inference on all test images
print("\n🔍 Running detection on test images...")
results = model.predict(
    source=working_test_dir,
    conf=0.25,  # Confidence threshold
    iou=0.45,   # NMS IoU threshold
    imgsz=640,   # Inference size
    save=True,   # Save results
    save_txt=True,  # Save results as .txt
    save_conf=True,  # Save confidence scores
    project="/kaggle/working/runs/detect",  # Save location
    name="predict_local_&_zenodo",  # Subfolder name
    exist_ok=True  # Overwrite existing files
)

print("\n🎯 Detection complete! Results saved to:")
print("/kaggle/working/runs/detect/predict")

🔄 Converting test images to BMP...
✅ Converted 30 test images to BMP format

🔍 Running detection on test images...

image 1/30 /kaggle/working/test_samples/exported_grid_test_1.bmp: 640x640 2 solar_panels, 47.3ms
image 2/30 /kaggle/working/test_samples/exported_grid_test_10.bmp: 640x640 2 solar_panels, 47.3ms
image 3/30 /kaggle/working/test_samples/exported_grid_test_11.bmp: 640x640 4 solar_panels, 45.7ms
image 4/30 /kaggle/working/test_samples/exported_grid_test_12.bmp: 640x640 3 solar_panels, 27.6ms
image 5/30 /kaggle/working/test_samples/exported_grid_test_13.bmp: 640x640 4 solar_panels, 27.1ms
image 6/30 /kaggle/working/test_samples/exported_grid_test_14.bmp: 640x640 2 solar_panels, 27.0ms
image 7/30 /kaggle/working/test_samples/exported_grid_test_15.bmp: 640x640 (no detections), 27.1ms
image 8/30 /kaggle/working/test_samples/exported_grid_test_16.bmp: 640x640 4 solar_panels, 26.7ms
image 9/30 /kaggle/working/test_samples/exported_grid_test_17.bmp: 640x640 8 solar_panels, 25.8ms
im